In [1]:
import load_data
df = load_data.load_county_level(data_dir='data')

loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [2]:
for name in df.keys():
    print(name)

countyFIPS
STATEFP
COUNTYFP
CountyName
StateName
State
lat
lon
POP_LATITUDE
POP_LONGITUDE
CensusRegionName
CensusDivisionName
Rural-UrbanContinuumCode2013
PopulationEstimate2018
PopTotalMale2017
PopTotalFemale2017
FracMale2017
PopulationEstimate65+2017
PopulationDensityperSqMile2010
CensusPopulation2010
MedianAge2010
#EligibleforMedicare2018
MedicareEnrollment,AgedTot2017
3-YrDiabetes2015-17
DiabetesPercentage
HeartDiseaseMortality
StrokeMortality
Smokers_Percentage
RespMortalityRate2014
#FTEHospitalTotal2017
TotalM.D.'s,TotNon-FedandFed2017
#HospParticipatinginNetwork2017
#Hospitals
#ICU_beds
dem_to_rep_ratio
PopMale<52010
PopFmle<52010
PopMale5-92010
PopFmle5-92010
PopMale10-142010
PopFmle10-142010
PopMale15-192010
PopFmle15-192010
PopMale20-242010
PopFmle20-242010
PopMale25-292010
PopFmle25-292010
PopMale30-342010
PopFmle30-342010
PopMale35-442010
PopFmle35-442010
PopMale45-542010
PopFmle45-542010
PopMale55-592010
PopFmle55-592010
PopMale60-642010
PopFmle60-642010
PopMale65-742010
P

In [3]:
keys = df.keys()
for i, val in enumerate(df.iloc[0]):
    print(i, '\t', keys[i], '\t', val)

0 	 countyFIPS 	 36047
1 	 STATEFP 	 36.0
2 	 COUNTYFP 	 47.0
3 	 CountyName 	 Kings
4 	 StateName 	 NY
5 	 State 	 New York
6 	 lat 	 40.64057761
7 	 lon 	 -73.95465424
8 	 POP_LATITUDE 	 40.650523
9 	 POP_LONGITUDE 	 -73.95481099999998
10 	 CensusRegionName 	 Northeast
11 	 CensusDivisionName 	 Middle Atlantic
12 	 Rural-UrbanContinuumCode2013 	 1.0
13 	 PopulationEstimate2018 	 2582830.0
14 	 PopTotalMale2017 	 1254966.0
15 	 PopTotalFemale2017 	 1393805.0
16 	 FracMale2017 	 0.4737918075968062
17 	 PopulationEstimate65+2017 	 356714.0
18 	 PopulationDensityperSqMile2010 	 35369.2
19 	 CensusPopulation2010 	 2504700.0
20 	 MedianAge2010 	 34.1
21 	 #EligibleforMedicare2018 	 386673.0
22 	 MedicareEnrollment,AgedTot2017 	 309657.0
23 	 3-YrDiabetes2015-17 	 635.0
24 	 DiabetesPercentage 	 9.7
25 	 HeartDiseaseMortality 	 195.8
26 	 StrokeMortality 	 19.5
27 	 Smokers_Percentage 	 13.840801998
28 	 RespMortalityRate2014 	 23.05
29 	 #FTEHospitalTotal2017 	 35442.0
30 	 TotalM.D.'s,Tot

## Build Graphs

### Contiguous graph

Counties are connected if they are next to each other.

In [4]:
import numpy as np

county_names = df['CountyName']
county_code = np.array([int(code) for code in df['countyFIPS']])
county_code_dict = {code:idx for idx, code in enumerate(county_code)}

In [5]:
import pandas as pd
data_contiguous = pd.read_csv('/home/ubuntu/covid19-severity-prediction/data/neighborcounties.csv')
df_contiguous = pd.DataFrame(data_contiguous, columns= ['orgfips','adjfips','instate'])

In [6]:
import dgl

g = dgl.DGLGraph()
g.add_nodes(len(county_code_dict))
for ind in df_contiguous.index: 
    src = int(df_contiguous['orgfips'][ind])
    dst = int(df_contiguous['adjfips'][ind])
    if src in county_code_dict and dst in county_code_dict:
        g.add_edge(county_code_dict[src],county_code_dict[dst])

Using backend: pytorch


### Load attributes of counties

In [7]:
import numpy as np
import torch as th
attr_cols = []

# Set the variable to True if we want to specify features ourselves.
use_specified_fields = False

if use_specified_fields:
    f = open("fields.txt", "r")
    fields = f.readlines()
    for field in fields:
        field = field.rstrip()
        arr = np.array(df[field])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))
else:
    for i in range(6, 86):
        arr = np.array(df[keys[i]])
        if arr.dtype == np.float32 or arr.dtype == np.float64:
            attr_cols.append(np.array(arr, dtype=np.float32))

attrs = th.tensor(np.stack(attr_cols, 1), dtype=th.float32)
cases = []
for i in range(87, 216):
    cases.append(np.array(df[keys[i]]))
cases = np.stack(cases, 1)
deaths = []
for i in range(218, 347):
    deaths.append(np.array(df[keys[i]]))
deaths = np.stack(deaths, 1)
print('attrs:', attrs.shape)
print('cases:', cases.shape)
print('deaths:', deaths.shape)
print(attrs[0])
print(cases[0])
print(deaths[0])

attrs: torch.Size([3142, 78])
cases: (3142, 129)
deaths: (3142, 129)
tensor([ 4.0641e+01, -7.3955e+01,  4.0651e+01, -7.3955e+01,  1.0000e+00,
         2.5828e+06,  1.2550e+06,  1.3938e+06,  4.7379e-01,  3.5671e+05,
         3.5369e+04,  2.5047e+06,  3.4100e+01,  3.8667e+05,  3.0966e+05,
         6.3500e+02,  9.7000e+00,  1.9580e+02,  1.9500e+01,  1.3841e+01,
         2.3050e+01,  3.5442e+04,  9.3070e+03,  6.0000e+00,  1.2000e+01,
         3.1800e+02,  4.5415e+00,  9.0415e+04,  8.6783e+04,  8.1356e+04,
         7.8035e+04,  7.9604e+04,  7.6959e+04,  8.6716e+04,  8.3968e+04,
         9.6663e+04,  9.9134e+04,  1.0605e+05,  1.1679e+05,  9.7154e+04,
         1.0570e+05,  1.6196e+05,  1.7958e+05,  1.4949e+05,  1.7469e+05,
         6.3789e+04,  7.8116e+04,  5.5241e+04,  6.8866e+04,  6.4628e+04,
         8.8066e+04,  3.5780e+04,  5.8209e+04,  1.2524e+04,  2.8426e+04,
         1.5400e+02,  2.4000e+01,  3.3000e+01,  1.5100e+02,  3.4100e+02,
         4.9300e+02,  1.1160e+03,  2.1390e+03,  2.8520e

### Data split: Give 20 days as features

In [8]:
import torch as th

num_days = cases.shape[1]
num_test_days = 15
train_test_split = num_days - num_test_days
# number of days for input
in_len = 20
# number of days for output
out_len = 7


train_attr_idxs = []
train_past_cases = []
train_labels_cases = []
train_labels_deaths = []
for start in range(train_test_split - in_len - out_len + 1):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    train_attr_idxs.append(np.arange(len(cases)))
    train_past_cases.append(cases[:,start:end_begin])
    train_labels_cases.append(cases[:, end_begin: end])
    train_labels_deaths.append(deaths[:, end_begin: end])
# num_seqs = len(train_past_cases)
# print(num_seqs)


test_attr_idxs = []
test_past_cases = []
test_labels_cases = []
test_labels_deaths = []
for start in range(train_test_split - in_len - out_len + 1,
                    num_days - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    test_attr_idxs.append(np.arange(len(cases)))
    test_past_cases.append(cases[:,start:end_begin])
    test_labels_cases.append(cases[:,end_begin: end])
    test_labels_deaths.append(deaths[:,end_begin: end])

    
train_attr_idxs = np.concatenate(train_attr_idxs, axis=0)
train_past_cases = th.tensor(np.concatenate(train_past_cases, axis=0), dtype=th.float32)
train_labels_cases = th.tensor(np.concatenate(train_labels_cases, axis=0), dtype=th.float32)
train_labels_deaths = th.tensor(np.concatenate(train_labels_deaths, axis=0), dtype=th.float32)
test_attr_idxs = np.concatenate(test_attr_idxs, axis=0)
test_past_cases = th.tensor(np.concatenate(test_past_cases, axis=0), dtype=th.float32)
test_labels_cases = th.tensor(np.concatenate(test_labels_cases, axis=0), dtype=th.float32)
test_labels_deaths = th.tensor(np.concatenate(test_labels_deaths, axis=0), dtype=th.float32)
print(train_attr_idxs.shape)
print(test_attr_idxs.shape)
print('train:', train_past_cases.shape)
print('test:', test_past_cases.shape)
# print(test_labels_cases.shape)
# print(test_labels_deaths[0])

(276496,)
(43988,)
train: torch.Size([276496, 19])
test: torch.Size([43988, 19])


## Loss 
### Mean Square Log Error

In [9]:
def my_msle(output, target):
    loss = th.mean(((output.add(1)).log() - (target.add(1)).log())**2)
    return loss

## MLP model¶

In [10]:
def init_weights(m):
    if type(m) == nn.Linear:
        th.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [11]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.nn_model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            #nn.BatchNorm1d(num_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.nn_model(x)

In [12]:
import torch.nn as nn

class Net_SIR(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net_SIR, self).__init__()
        self.nn_model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            #nn.BatchNorm1d(num_features=hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.nn_model(x)

### MLP training loop

In [13]:
hidden_size = 50
out_size = 7
learning_rate = 0.000001
num_epochs = 200
batch_size = 1000

print('mean of training labels: {:.3f}, mean of testing labels: {:.3f}'.format(th.mean(train_labels_cases),
                                                                               th.mean(test_labels_cases)))

device = th.device('cuda:0')
input_size = train_past_cases.shape[1] + attrs.shape[1]
net = Net(input_size, hidden_size, out_size)
# net.apply(init_weights)
# criterion = nn.MSELoss()
print('#features:', input_size)

# Move model to GPU
net = net.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
train_past_cases = train_past_cases.to(device)
train_labels_cases = train_labels_cases.to(device)
train_labels_deaths = train_labels_deaths.to(device)
if len(train_labels_cases.shape) == 1:
    train_labels_cases = train_labels_cases.unsqueeze(1)
if len(train_labels_deaths.shape) == 1:
    train_labels_deaths = train_labels_deaths.unsqueeze(1)
test_past_cases = test_past_cases.to(device)
test_labels_cases = test_labels_cases.to(device)
test_labels_deaths = test_labels_deaths.to(device)
if len(test_labels_cases.shape) == 1:
    test_labels_cases = test_labels_cases.unsqueeze(1)
if len(test_labels_deaths.shape) == 1:
    test_labels_deaths = test_labels_deaths.unsqueeze(1)
print(train_labels_cases.shape)

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    sample_idxs = th.tensor(range(len(train_past_cases)))
    batch_idxs = th.split(sample_idxs, batch_size)
    losses = []
    for idx in batch_idxs:
        attr_idx = train_attr_idxs[idx]
        labels = train_labels_cases[idx]
        labels[th.isnan(labels)] = 0
        batch = th.cat([train_past_cases[idx], attrs[attr_idx]], dim=1)
        vals = net(batch)
        
        loss = my_msle(vals, labels)
        # loss = criterion(vals, labels)
        # optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        attr_idx = test_attr_idxs
        test_feats = th.cat([test_past_cases, attrs[attr_idx]], dim=1)
        eval_vals = net(test_feats)
        
        # eval_rmse = th.mean(th.abs(eval_vals - test_labels_cases))
        eval_rmse = my_msle(eval_vals, test_labels_cases)
    print('epoch={}, loss={:.3f}, test loss={:.3f}'.format(epoch, np.mean(losses), eval_rmse))

mean of training labels: 117.055, mean of testing labels: 470.353
#features: 97
torch.Size([276496, 7])
epoch=0, loss=4.153, test loss=10.249
epoch=1, loss=3.912, test loss=9.505
epoch=2, loss=3.781, test loss=9.043
epoch=3, loss=3.679, test loss=8.681
epoch=4, loss=3.589, test loss=8.367
epoch=5, loss=3.507, test loss=8.080
epoch=6, loss=3.429, test loss=7.816
epoch=7, loss=3.356, test loss=7.570
epoch=8, loss=3.286, test loss=7.340
epoch=9, loss=3.218, test loss=7.121
epoch=10, loss=3.153, test loss=6.910
epoch=11, loss=3.089, test loss=6.707
epoch=12, loss=3.027, test loss=6.509
epoch=13, loss=2.966, test loss=6.316
epoch=14, loss=2.906, test loss=6.128
epoch=15, loss=2.847, test loss=5.943
epoch=16, loss=2.788, test loss=5.760
epoch=17, loss=2.730, test loss=5.580
epoch=18, loss=2.672, test loss=5.404
epoch=19, loss=2.614, test loss=5.231
epoch=20, loss=2.557, test loss=5.061
epoch=21, loss=2.500, test loss=4.895
epoch=22, loss=2.444, test loss=4.733
epoch=23, loss=2.390, test loss

## GNN

### Prepare data

In [16]:
import torch as th

num_days = cases.shape[1]
num_test_days = 15
train_test_split = num_days - num_test_days
in_len = 20
out_len = 7


train_past_cases = []
train_past_deaths = []
train_labels_cases = []
train_labels_deaths = []
for start in range(train_test_split - in_len - out_len + 1):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    train_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    train_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    train_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    train_labels_deaths.append(th.tensor(deaths[:,end_begin:end], dtype=th.float32))

test_past_cases = []
test_past_deaths = []
test_labels_cases = []
test_labels_deaths = []
for start in range(train_test_split - in_len - out_len + 1,
                    num_days - in_len - out_len):
    end = start + in_len + out_len - 1
    end_begin = end - out_len
    test_past_cases.append(th.tensor(cases[:,start:end_begin], dtype=th.float32))
    test_past_deaths.append(th.tensor(deaths[:,start:end_begin], dtype=th.float32))
    test_labels_cases.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))
    test_labels_deaths.append(th.tensor(cases[:,end_begin:end], dtype=th.float32))

### GAT - MSLE

In [17]:
from dgl.nn import pytorch as dglnn
import torch.nn.functional as F
import torch.nn as nn

class GAT(nn.Module):
    def __init__(self, g, input_size, hidden_size, gcn_nlayers, num_heads=5):
        super(GAT, self).__init__()
        self.g = g
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(dglnn.conv.GATConv(input_size, hidden_size, num_heads=num_heads,
                                                  residual=True, activation=F.relu))
        for i in range(gcn_nlayers):
            self.gcn_layers.append(dglnn.conv.GATConv(num_heads * hidden_size, hidden_size, num_heads=num_heads,
                                                      residual=True, activation=F.relu))

        self.linear_layers = nn.Sequential(
            nn.Linear(num_heads * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.ReLU()
        )

    def forward(self, x):
        h = x
        for layer in self.gcn_layers:
            h = layer(self.g, h).flatten(1)
        return self.linear_layers(h)

In [ ]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 500
batch_size = 1000

device = th.device('cuda:0')
net = GAT(g, train_past_cases[0].shape[1] + attrs.shape[1], hidden_size, 1)
# criterion = nn.MSELoss()
print('#features:', train_past_cases[0].shape[1] + attrs.shape[1])

# Move model to GPU
net = net.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        #batch = train_past_cases[idx]
        vals = net(batch)
        # loss = criterion(vals, labels)
        loss = my_msle(vals, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        eval_errs = []
        for cases, labels in zip(test_past_cases, test_labels_cases):
            test_feats = th.cat([cases, attrs], dim=1)
            #test_feats = test_past_cases
            eval_vals = net(test_feats)
            err = my_msle(eval_vals, labels)
            # err = th.mean(th.abs(eval_vals - labels))
            eval_errs.append(err.cpu().numpy())
            #print(eval_vals, test_labels)
    print('epoch={}, loss={:.3f}, test loss={:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs)))

#features: 97


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


epoch=0, loss=5.647, test loss=16.902
epoch=1, loss=5.643, test loss=16.894
epoch=2, loss=5.506, test loss=16.695
epoch=3, loss=4.805, test loss=14.819
epoch=4, loss=2.674, test loss=5.014
epoch=5, loss=1.422, test loss=2.459
epoch=6, loss=1.220, test loss=2.173
epoch=7, loss=1.139, test loss=2.116
epoch=8, loss=1.106, test loss=2.150
epoch=9, loss=1.094, test loss=2.223
epoch=10, loss=1.095, test loss=2.312
epoch=11, loss=1.102, test loss=2.393
epoch=12, loss=1.110, test loss=2.472
epoch=13, loss=1.120, test loss=2.545
epoch=14, loss=1.127, test loss=2.605
epoch=15, loss=1.132, test loss=2.658
epoch=16, loss=1.134, test loss=2.698
epoch=17, loss=1.131, test loss=2.728
epoch=18, loss=1.126, test loss=2.754
epoch=19, loss=1.118, test loss=2.775
epoch=20, loss=1.112, test loss=2.792
epoch=21, loss=1.107, test loss=2.804
epoch=22, loss=1.102, test loss=2.808
epoch=23, loss=1.097, test loss=2.808
epoch=24, loss=1.089, test loss=2.799
epoch=25, loss=1.078, test loss=2.773
epoch=26, loss=1.0

epoch=213, loss=0.180, test loss=0.351
epoch=214, loss=0.175, test loss=0.320
epoch=215, loss=0.176, test loss=0.338
epoch=216, loss=0.182, test loss=0.359
epoch=217, loss=0.174, test loss=0.318
epoch=218, loss=0.181, test loss=0.368
epoch=219, loss=0.185, test loss=0.382
epoch=220, loss=0.178, test loss=0.339
epoch=221, loss=0.175, test loss=0.333
epoch=222, loss=0.185, test loss=0.391
epoch=223, loss=0.174, test loss=0.315
epoch=224, loss=0.176, test loss=0.343
epoch=225, loss=0.184, test loss=0.381
epoch=226, loss=0.180, test loss=0.347
epoch=227, loss=0.173, test loss=0.317
epoch=228, loss=0.187, test loss=0.387
epoch=229, loss=0.173, test loss=0.315
epoch=230, loss=0.177, test loss=0.352
epoch=231, loss=0.185, test loss=0.392
epoch=232, loss=0.182, test loss=0.372
epoch=233, loss=0.176, test loss=0.346
epoch=234, loss=0.181, test loss=0.381
epoch=235, loss=0.182, test loss=0.370
epoch=236, loss=0.170, test loss=0.310
epoch=237, loss=0.180, test loss=0.367
epoch=238, loss=0.184, te

epoch=424, loss=0.204, test loss=0.509
epoch=425, loss=0.227, test loss=0.591
epoch=426, loss=0.231, test loss=0.593
epoch=427, loss=0.215, test loss=0.513


### GAT - SIR

In [20]:
pop = attrs[:,13]
pop = pop.view(3142, 1)
pop = pop.to(device)

In [15]:
from dgl.nn import pytorch as dglnn
import torch.nn.functional as F
import torch.nn as nn

class GAT_SIR(nn.Module):
    def __init__(self, g, input_size, hidden_size, output_size, gcn_nlayers, num_heads=5):
        super(GAT_SIR, self).__init__()
        self.g = g
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(dglnn.conv.GATConv(input_size, hidden_size, num_heads=num_heads,
                                                  residual=True, activation=F.relu))
        for i in range(gcn_nlayers):
            self.gcn_layers.append(dglnn.conv.GATConv(num_heads * hidden_size, hidden_size, num_heads=num_heads,
                                                      residual=True, activation=F.relu))

        self.linear_layers = nn.Sequential(
            nn.Linear(num_heads * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        h = x
        for layer in self.gcn_layers:
            h = layer(self.g, h).flatten(1)
        return self.linear_layers(h)

In [41]:
"""
Input: beta(from NN), gamma(from NN), N(from attributes)
Output: mean square log error for infected cases
"""
def sir_loss(params, I, D, target):
    I[I != I] = 0.0
    D[D != D] = 0.0
    target[target != target] = 0.0
    err = th.tensor([0],dtype=th.float)
    err = err.to(device)
    S = pop - I - D
    l = int(len(params[0]) / 2)
    for i in range(l):
        S_new = th.add(S, - th.div(th.mul(params[:,i].view(3142,1), th.mul(S, I)), pop))
        I_new = th.add(th.add(I, th.mul(params[:,i].view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(params[:,i + l].view(3142,1), I))
        D_new = th.add(D, th.mul(params[:,i].view(3142,1), I))
        temp = (I_new - target[:,i].view(3142,1)).pow(2)
        print(th.mean(temp))
        err = err + th.mean(temp)
        S = S_new
        I = I_new
        D = D_new
    print(err)
    return err

In [ ]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 500
batch_size = 1000

device = th.device('cuda:0')
input_size = train_past_cases[0].shape[1] + attrs.shape[1]
output_size = 14
net = GAT_SIR(g, input_size, hidden_size, output_size, 1)
print('#features:', input_size)

# Move model to GPU
net = net.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
pop = pop.to(device)
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_past_deaths[i] = train_past_deaths[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_past_deaths[i] = test_past_deaths[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    # sample_idxs = th.randperm(len(train_past_cases))
    losses = []
    for idx in range(len(train_past_cases)):
        labels = train_labels_cases[idx]
        batch = th.cat([train_past_cases[idx], attrs], dim=1)
        vals = net(batch)
        print(vals)
        # params = th.cat((vals,pop), dim = 1) # shape = 3142 * 3
        I = train_past_cases[idx][:,-1].view(3142,1)
        D = train_past_deaths[idx][:,-1].view(3142,1)
        loss = sir_loss(vals, I, D, labels)
        # print(loss)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    
    with th.no_grad():
        eval_errs = []
        for cases, deaths, labels in zip(test_past_cases, test_past_deaths, test_labels_cases):
            test_feats = th.cat([cases, attrs], dim=1)
            #test_feats = test_past_cases
            eval_vals = net(test_feats)
            eval_I = cases[:,-1].view(3142,1)
            eval_D = deaths[:, -1].view(3142,1)
            err = sir_loss(eval_vals, eval_I, eval_D, labels)
            # err = th.mean(th.abs(eval_vals - labels))
            eval_errs.append(err.cpu().numpy())
            #print(eval_vals, test_labels)
    print('epoch={}, loss={:.3f}, test loss={:.3f}'.format(epoch, np.mean(losses), np.mean(eval_errs)))

#features: 97
tensor([[0.5112, 0.4688, 0.5234,  ..., 0.5201, 0.5189, 0.5072],
        [0.5118, 0.4712, 0.5226,  ..., 0.5203, 0.5168, 0.5065],
        [0.5099, 0.4722, 0.5222,  ..., 0.5211, 0.5149, 0.5118],
        ...,
        [0.4971, 0.4595, 0.5332,  ..., 0.4901, 0.5194, 0.5434],
        [0.5030, 0.4597, 0.5337,  ..., 0.4910, 0.5304, 0.5347],
        [0.4973, 0.4592, 0.5325,  ..., 0.4901, 0.5188, 0.5431]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor(0.1765, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(15401.8340, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(2.8588e+14, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(8.1176e+34, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(inf, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

...

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([nan], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

In [42]:
hidden_size = 100
learning_rate = 0.0000005
num_epochs = 500
batch_size = 1000

device = th.device('cuda:0')
input_size = train_past_cases[0].shape[1] + attrs.shape[1]
output_size = 2
net = GAT_SIR(g, input_size, hidden_size, output_size, 1)
print('#features:', input_size)

# Move model to GPU
net = net.to(device)
# TODO we might want to choose a better loss.
# L2 loss will penalize the data points with larger deaths more. Is this what we want?
# criterion = criterion.to(device)

# Move data to GPU
pop = pop.to(device)
attrs = attrs.to(device)
attrs[th.isnan(attrs)] = 0
for i in range(len(train_past_cases)):
    train_past_cases[i] = train_past_cases[i].to(device)
    train_past_deaths[i] = train_past_deaths[i].to(device)
    train_labels_cases[i] = train_labels_cases[i].to(device)
    if len(train_labels_cases[i].shape) == 1:
        train_labels_cases[i] = train_labels_cases[i].unsqueeze(1)
for i in range(len(test_past_cases)):
    test_past_cases[i] = test_past_cases[i].to(device)
    test_past_deaths[i] = test_past_deaths[i].to(device)
    test_labels_cases[i] = test_labels_cases[i].to(device)
    if len(test_labels_cases[i].shape) == 1:
         test_labels_cases[i] = test_labels_cases[i].unsqueeze(1)

# Normalize attrs
attr_mean = th.mean(attrs, dim=1, keepdim=True)
attrs = attrs - attr_mean
attr_std = th.std(attrs, dim=1, keepdim=True)
attrs = attrs / attr_std

optimizer = th.optim.Adam(net.parameters(), lr=learning_rate)

sample_idxs = range((len(train_past_cases)))
losses = []
idx = sample_idxs[0]
labels = train_labels_cases[idx]
batch = th.cat([train_past_cases[idx], attrs], dim=1)
vals = net(batch)
# print(vals.shape)
I = train_past_cases[idx][:,-1].view(3142,1)
D = train_past_deaths[idx][:,-1].view(3142,1)
S = pop - I - D
# print(vals[:,0].shape)
loss = sir_loss(vals, I, D, labels)
print(loss)
loss.backward()

#features: 97
tensor(0.1544, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([0.1544], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1544], device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
params = vals
I = train_past_cases[idx][:,-1].view(3142,1)
D = train_past_deaths[idx][:,-1].view(3142,1)
S = pop - I - D
i = 0
S_new = th.add(S, - th.div(th.mul(params[:,i].view(3142,1), th.mul(S, I)), pop))
print(S_new)
print(S.shape)
print(th.mean(S))

In [ ]:
l = 1
i = 0
I_new = th.add(th.add(I, th.mul(params[:,i].view(3142,1), th.div(th.mul(S,I), pop))), - th.mul(params[:,i + l].view(3142,1), I))
print(I_new)
print(I_new.shape)
print(th.mean(I_new))